# Různé scénáře využití RLS
Následující ukázky pracují s dalšími podmínkami v security predikátu a s použitím kontextu připojení.

Předpoklady úspěšného fungování:
- vykonání notebooku RLS_1
- vykonání notebooku RLS_2


In [ ]:
use RLS
go

drop table if EXISTS dbo.AdvancedRls, dbo.Dochazka

create table dbo.AdvancedRls
(
    Id int not null identity constraint pk_AdvancedRls primary key
    , Jmeno nvarchar(30) not null
    , Bohatstvi int not null
    , OsobniCislo nchar(3) not null
)
go

insert dbo.AdvancedRls values
('Jiří', 1000, '001')
, ('Jakub', 200, '001')
, ('Vladimír', 1100, '001')
, ('Jana', 1000, '002')
go

create table dbo.Dochazka
(
    Id int not null identity constraint pk_Dochazka primary key
    , OsobniCislo nchar(3) not null
    , Prichod datetime2 not NULL
    , Odchod datetime2 null
)
go

insert dbo.Dochazka (OsobniCislo, Prichod, Odchod) values ('001', '2023-05-25 7:00', null)
go

Následující příklad nám pomůže vymyslet, jak má vypadat security predikát.

In [ ]:
use RLS
go

select 1 as col1 from dbo.Dochazka where SYSDATETIME() between Prichod and coalesce(Odchod, dateadd(mm, 1, SYSDATETIME())) and OsobniCislo = '001'

Hodnota osobního čísla zadaná v poslední podmínce bude tvořit parametr funkce. Problém je, že osobní číslo není nijak automaticky zjistitelná hodnota. Musí se nějak přidat do kontextu připojení. Na to slouží **CONTEXT\_INFO**. Je to jednak vlastnost připojení, a jednak funkce, která umí hodnotu přečíst **CONTEXT\_INFO** je vždy binární řetězec. Nejprve si ukážeme, jak funguje nastavení a přečtení hodnoty.

In [ ]:
use RLS
go

declare @context varbinary(16) = cast('abc' as varbinary)

set context_info @context

select cast(CONTEXT_INFO() as char(3))

A teď zkombinujeme **CONTEXT\_INFO** a nachystaný příkaz **SELECT** dohromady ve funkci.

In [ ]:
use RLS
go

create or alter function rls.fnFiltrAdvanced(@osobniCislo nchar(3))
returns table
with SCHEMABINDING
as
return
select 1 as col1 from dbo.Dochazka where SYSDATETIME() between Prichod and coalesce(Odchod, dateadd(minute, 1, SYSDATETIME())) and @osobniCislo = cast(CONTEXT_INFO() as char(3))
GO

Vytvoříme politiku...

In [ ]:
use RLS
go

create security policy rls.policyAdvancedRls
add filter predicate rls.fnFiltrAdvanced(OsobniCislo) on dbo.AdvancedRls
with (state = on)
go

... a celý příklad otestujeme.

In [ ]:
use RLS
go
set context_info -1
go
select * from dbo.AdvancedRls
GO

declare @context varbinary(16) = cast('001' as varbinary)

set context_info @context

select * from dbo.AdvancedRls
go 

Dokončená ukázka neřeší případ uživatele, který může vidět všechna data. V praxi jsem pro takové uživatele použil nějakou domluvenou hodnotu vlastnosti CONTEXT\_INFO a doplnění podmínky do security predikátu.